# Path Optimization (KOMO)



## Demo of features in Inverse Kinematics

Let's setup a standard configuration. (Lock the window with "Always on Top".)

In [1]:
import sys
sys.path.append('../ry')

from libry import *
from numpy import *

K = Config()
D = K.view()
K.addFile('../rai-robotModels/pr2/pr2.g')
K.addFile('../test/kitchen.g')

Let's add some objects

In [2]:
K.addFrame('item1', 'sink1', 'type:ssBox Q:<t(-.1 -.1 .52)> size:[.1 .1 .25 .02] color:[1. 0. 0.], contact' )
K.addFrame('item2', 'sink1', 'type:ssBox Q:<t(.1 .1 .52)> size:[.1 .1 .25 .02] color:[1. 1. 0.], contact' )
K.addFrame('tray', 'stove1', 'type:ssBox Q:<t(.0 .0 .42)> size:[.2 .2 .05 .02] color:[0. 1. 0.], contact' )

180

compute a collision free path to touch object item1 with pr2L:

In [3]:
X0 = K.getFrameState()

obj1 = "item1";
arm = "pr2R";

komo = K.komo_path(1.,20, 10.);

komo.addObjective([], OT.sos, FS.transAccelerations, [], [1.]);
komo.addObjective([], OT.eq, FS.accumulatedCollisions);
komo.addObjective([], OT.ineq, FS.jointLimits);
komo.addObjective([1.], OT.eq, FS.distance, [arm, obj1]);
komo.addObjective([.9,1.], OT.sos, FS.positionDiff, ["endeffWorkspace", obj1], [1e0]);
komo.addObjective(time=[1.], type=OT.eq, feature=FS.qItself, order=1);

komo.optimize();
komo.getReport()

[{'description': 'QuaternionNorms',
  'scale': 3.0,
  'sos_value': 0.0,
  'type': 'sos'},
 {'feature': 'Transition',
  'scale': 1.0,
  'sos_value': 3.049684090003639,
  'type': 'sos'},
 {'eq_sumOfAbs': 8.077365490100422e-05,
  'feature': 'ProxyCost',
  'scale': 10.0,
  'type': 'eq'},
 {'feature': 'LimitsConstraint', 'scale': 10.0, 'type': 'ineq'},
 {'eq_sumOfAbs': 0.0013584651687355738,
  'feature': 'dist',
  'o1': 'pr2R',
  'o2': 'item1',
  'scale': 10.0,
  'type': 'eq'},
 {'feature': 'posDiff',
  'o1': 'endeffWorkspace',
  'o2': 'item1',
  'scale': 1.0,
  'sos_value': 0.6146308342436854,
  'type': 'sos'},
 {'description': 'qItself:ALL',
  'eq_sumOfAbs': 0.004478496369865968,
  'scale': 10.0,
  'type': 'eq'}]

In [4]:
komo.displayTrajectory()

Best way to read out: grab a configuration into K and analyze it there:

In [5]:
K.setFrameState( komo.getConfiguration(19) )
K.getJointState()

array([-0.74942162,  1.18847244,  0.74510293,  0.10001565,  0.00236768,
       -0.00243315,  0.05870029,  1.1178125 ,  0.39325715,  0.37448196,
        0.44698932, -1.13291216,  0.96331118, -1.32398169, -2.07202749,
       -1.6244598 ,  1.49525196, -0.35612166, -0.49339393, -0.49966231,
        0.50107843,  0.09719817,  0.09590853,  0.01103211,  0.00950933])